In [131]:
from PIL import Image
import pandas as pd
import os

In [132]:

class MakeLocateDataFrame():
    
    def __init__(self, type: bool):

        self.type = type
        self.root = r'/opt/ml/data/'
        if self.type:
            self.root += 'train'
        else:
            self.root += 'eval'

    def get_csv(self):
        datas = None

        if self.type:
            datas = pd.read_csv(self.root+'/train.csv')
            datas = datas.drop(['race','id'],axis=1)

        else:
            datas = pd.read_csv(self.root+'/info.csv')

        return datas

    def make_image_locate_frame(self, datas):
        
        labeled_dict = None
        #train = True 인 경우
        if self.type:
            # 이미지 경로를 담은 리스트 생성
            image_locate = [self.root+'/images/'+i for i in datas['path']]
            images = []
            # 경로 안에 있는 숨김 파일은 제거하고 이미지만 저장
            for i in image_locate:
                temp = []
                for j in os.listdir(i):
                    if not j[0] == '.':
                        temp.append(i+'/'+j)
                images.append(temp)
            # 파일 이름으로 클래스를 구분하기 위한 문자
            file_name_string = 'min'
            #새롭게 반환할 데이터 프레임의 틀
            labeled_dict = {"label": [],
                            "gender": [],
                            "age": [],
                            "locate": []}

            for image in images:
                for j in image:
                    #/opt/ml/data/train/images/000002_female_Asian_52/normal.jpg 자른 것
                    get_locate_file = j.split('/')
                    #가장 끝의 파일은 이미지 이름이다.
                    label = file_name_string.index(get_locate_file[-1][0])
                    start = label * 6
                    #000002_female_Asian_52 잘라서 굳이 위에 선언한 데이터 불러오지 않고 수행
                    _, sex, _, age = get_locate_file[-2].split('_')
                    age = int(age)
                    if sex == 'female':
                        start +=3
                    if age < 30:
                        pass
                    elif age < 60:
                        start +=1
                    else:
                        start +=2
                    labeled_dict['label'].append(start)
                    labeled_dict['gender'].append(sex)
                    labeled_dict['age'].append(age)
                    labeled_dict['locate'].append(j)

            labeled_dict = pd.DataFrame(labeled_dict, columns=labeled_dict.keys())

        else:
            datas['locate'] = [self.root+'/images/'+i for i in datas['ImageID']]
            labeled_dict = datas

        return labeled_dict

In [133]:
a = MakeLocateDataFrame(True)

data = a.get_csv()
# data
loc = a.make_image_locate_frame(data)
data = a.make_image_locate_frame(data)


print(f"30대 이전:\n{data.query(f'age < 30').count()}")
print(f"60대 이전: \n{data.query(f'(age < 60) & (age >=30)').count()}")
print(f"60대: \n{data.query(f'age == 60').count()}")

over_60 = data.query('age==60')


30대 이전:
label     8967
gender    8967
age       8967
locate    8967
dtype: int64
60대 이전: 
label     8589
gender    8589
age       8589
locate    8589
dtype: int64
60대: 
label     1344
gender    1344
age       1344
locate    1344
dtype: int64


In [134]:
img = Image.open(over_60.iloc[15]['locate'])
print(type(over_60))
print(over_60.iloc[15]['locate'])
over_60
# img
#apply lambda -> 쓰면 내가 원하는 동작 될거같다잉.


<class 'pandas.core.frame.DataFrame'>
/opt/ml/data/train/images/001047_male_Asian_60/mask4.jpg


,label,gender,age,locate
2912,2,male,60,/opt/ml/data/train/images/001038_male_Asian_60...
2913,2,male,60,/opt/ml/data/train/images/001038_male_Asian_60...
2914,8,male,60,/opt/ml/data/train/images/001038_male_Asian_60...
2915,2,male,60,/opt/ml/data/train/images/001038_male_Asian_60...
2916,2,male,60,/opt/ml/data/train/images/001038_male_Asian_60...
...,...,...,...,...
15521,11,female,60,/opt/ml/data/train/images/005515_female_Asian_...
15522,5,female,60,/opt/ml/data/train/images/005515_female_Asian_...
15523,5,female,60,/opt/ml/data/train/images/005515_female_Asian_...
15524,5,female,60,/opt/ml/data/train/images/005515_female_Asian_...


In [135]:
from matplotlib import pyplot as plt
import albumentations as A
import cv2
# img = Image.open(over_60.iloc[15]['locate'])

ORIGIN_FILENAME = ("normal", "mask1", "mask2", "mask3", "mask4", "mask5", "incorrect_mask")
REWRITE_FILENAME = ("normal_hf", "mask1_hf", "mask2_hf", "mask3_hf", "mask4_hf", "mask5_hf", "incorrect_mask_hf")
file_rename = []
transform = A.Compose([
    # A.RandomCrop(width=256, height=256),
    A.HorizontalFlip(p=1),
    A.Cutout(num_holes=1,max_h_size=100, max_w_size=100,p=0.7)
])
for image_locate in over_60['locate']:
    img = cv2.imread(image_locate)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    image_transform = transform(image=img)
    # plt.imshow(a['image'])

    loc_split = image_locate.split('/')
    filename, extension = loc_split[-1].split('.')
    loc_split[-1] = REWRITE_FILENAME[ORIGIN_FILENAME.index(filename)]+'.'+extension
    rename_locate = '/'.join(loc_split)
    file_rename.append(rename_locate)
    # albumentation 처리된 후에 dict 로 반환되어 이미지만 따로 빼야함
    cv2.imwrite(filename=rename_locate, img=image_transform['image'])
# print(file_rename)
over_60['locate'] = file_rename
over_60.iloc[15]['locate']

'/opt/ml/data/train/images/001047_male_Asian_60/mask4_hf.jpg'

In [152]:
total_data = data.append(over_60)

total_data['ga'] = total_data['label'].astype(str) + total_data['gender']
total_data['race'] = ['Asian' for i in range(len(total_data))]
total_data.rename(columns={'locate':'path'}, inplace=True)
total_data.to_csv('train.csv', sep=',', index=False)
total_data


,label,gender,age,path,ga,race
0,4,female,45,/opt/ml/data/train/images/000001_female_Asian_...,4female,Asian
1,4,female,45,/opt/ml/data/train/images/000001_female_Asian_...,4female,Asian
2,10,female,45,/opt/ml/data/train/images/000001_female_Asian_...,10female,Asian
3,4,female,45,/opt/ml/data/train/images/000001_female_Asian_...,4female,Asian
4,4,female,45,/opt/ml/data/train/images/000001_female_Asian_...,4female,Asian
...,...,...,...,...,...,...
15521,11,female,60,/opt/ml/data/train/images/005515_female_Asian_...,11female,Asian
15522,5,female,60,/opt/ml/data/train/images/005515_female_Asian_...,5female,Asian
15523,5,female,60,/opt/ml/data/train/images/005515_female_Asian_...,5female,Asian
15524,5,female,60,/opt/ml/data/train/images/005515_female_Asian_...,5female,Asian


In [4]:
from torch.utils.data import Dataset
from PIL import Image

class MaskDataset(Dataset):
    def __init__(self, data, transform= None, train=True):
        self.data = data
        self.classes = self.data.columns.values.tolist()
        self.transform = transform
        self.train = train

    def __len__(self):
        return (len(self.data))

    def __getitem__(self, idx):
        X = Image.open(self.data['locate'].iloc[idx])

        if self.transform is not None:
            X = self.transform(X)
    
        if self.train:
            y = self.data['label'].iloc[idx]
            return X,y
        
        else:
            return X

In [23]:
import torch
torch.manual_seed(17)
import torchvision.transforms as transforms
from torchvision.transforms import *
from torch.utils.data import DataLoader
import albumentations as A
transform = transforms.Compose([
            # 선형 보간법을 사용하여 출력 픽셀값을 계산 -> 부드러운 느낌을 준다는데 누구 기준인진 잘
            transforms.Resize((512,384), Image.BILINEAR),
            transforms.ToTensor(),
            transforms.Normalize(0.5, 0.5),
            # transforms.Grayscale(num_output_channels=3),
            transforms.CenterCrop(384),
            A.Cutout(num_holes=1, max_h_size=100, max_w_size=100, p=0.7),
            ])
datasets = MaskDataset(data=a.make_image_locate_frame(data), transform=transform, train=True)
tf = transforms.ToPILImage()
img_t = tf(datasets[821][0])
print(datasets[1][0].shape)
img_t
# data_loader = Dataloader(datasets, num_workers=1, batch_size=128)


KeyError: 'You have to pass data to augmentations as named arguments, for example: aug(image=image)'

In [10]:
from torchvision import models
import torch.optim as optim

models = models.resnet18(pretrained=True)
optm = optim.Adam(models.parameters(), lr=0.001)
lr = optim.lr_scheduler.ReduceLROnPlateau(optm, mode='max',factor=0.5, patience=2)
print(lr.optimizer)
print(optm.state_dict()['param_groups'][0]['lr'])

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
0.001


In [157]:
loc.rename(columns={'locate':'path', 'label':'age_group'}, inplace=True)
loc['race'] = ['Asian' for i in range(len(loc))]
loc['ga'] = loc['age'].astype(str) + loc['gender']
loc.to_csv('train_dt.csv', sep=',', index=False)
loc

,age_group,gender,age,path,race,ga
0,4,female,45,/opt/ml/data/train/images/000001_female_Asian_...,Asian,45female
1,4,female,45,/opt/ml/data/train/images/000001_female_Asian_...,Asian,45female
2,10,female,45,/opt/ml/data/train/images/000001_female_Asian_...,Asian,45female
3,4,female,45,/opt/ml/data/train/images/000001_female_Asian_...,Asian,45female
4,4,female,45,/opt/ml/data/train/images/000001_female_Asian_...,Asian,45female
...,...,...,...,...,...,...
18895,6,male,19,/opt/ml/data/train/images/006959_male_Asian_19...,Asian,19male
18896,0,male,19,/opt/ml/data/train/images/006959_male_Asian_19...,Asian,19male
18897,0,male,19,/opt/ml/data/train/images/006959_male_Asian_19...,Asian,19male
18898,0,male,19,/opt/ml/data/train/images/006959_male_Asian_19...,Asian,19male


In [165]:
d = MakeLocateDataFrame(True)

def label(x:int):
    if x <30:
        return 0
    elif x <60:
        return 1
    else:
        return 2

go = d.get_csv()
go['race'] = ['Asian' for i in range(len(go))]
go['age_group'] = go['age'].map(lambda x: label(x))
go['ga'] = go['age'].astype(str) + go['gender']
go.to_csv('train_dt.csv', sep=',', index=False)
go

,gender,age,path,race,age_group,ga
0,female,45,000001_female_Asian_45,Asian,1,45female
1,female,52,000002_female_Asian_52,Asian,1,52female
2,male,54,000004_male_Asian_54,Asian,1,54male
3,female,58,000005_female_Asian_58,Asian,1,58female
4,female,59,000006_female_Asian_59,Asian,1,59female
...,...,...,...,...,...,...
2695,male,19,006954_male_Asian_19,Asian,0,19male
2696,male,19,006955_male_Asian_19,Asian,0,19male
2697,male,19,006956_male_Asian_19,Asian,0,19male
2698,male,20,006957_male_Asian_20,Asian,0,20male
